In [1]:
import numpy as np
import pandas as pd
import re

from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [62]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [75]:
test.sample(10)

,id,comment_text
22908,2638ed7a68d3b934,""" \n\n ==Happy {{subst:PAGENAME}}'s Day!== \n ..."
92121,998f79cf3fa8090e,"==See also page== \n\n Yes, I could make one, ..."
137876,e67bc5c6d595e738,== UNACADEMICS DESCIDING PLANETARY FACTS == \n...
38302,3f9332a7704a2bc1,ﻣﺤﻤﺪ \n ﻣﺼﻄﻔﯽ ﺻﮧ ( ﮐﮭﺎﻧﮯ ﮐﺎ \n ﮔﺮﺍ ﮨﻮﺍ ﭨﮑﮍﺍ ﺩﺳ...
7460,0c88fcd6070d9f02,This conversation has turned out rather intere...
122827,cd1a9cd834bd87f8,""" \n\n ==Whiteboys== \n New International E..."
57296,5f53c7256cf3e7c6,""" \n :::You say that the word """"terminology"""" ..."
64085,6ac3ae4328c250c3,== Revolutionary == \n The International Socia...
41008,4416ab74d30c3e97,""" \n\n The reason I didn't respond to your ini..."
56257,5d92cc292ff530b3,""" \n ::::::::WP:SYNTH is about as basic as it ..."


### The most common word

In [ ]:
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text], ignore_index=True)

In [4]:
def get_words(text):
    result = re.split(r'\s+|\.', text.lower())
    return result

In [5]:
total = list()
for line in all_text:
    total.extend(get_words(line))

In [10]:
count = Counter(total)
count.most_common(3)

[('', 1097088), ('the', 902873), ('to', 534076)]

Какое слово встречается чаще всего в объединенном train и test датасете? - **'the'** 

### Увеличение параметра C в Logistic regression увеличивает или уменьшает степень регуляризации?

Увеличение параметра C в Logistic regression увеличивает или уменьшает степень регуляризации?

**C** – обратный коэффициент регуляризации (тот самый C в sklearn-реализации LogisticRegression)

Ответ: **Уменьшает**

### Crossvalidation

In [14]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

In [25]:
word_vectorizer = TfidfVectorizer() # TfidfVectorizer или CountVectorizer

word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

In [60]:
C = 3.2

classifier = LogisticRegression(C=C,random_state=7) 

scores= []

for class_name in class_names:
    
    train_target = train[class_name]

    cv_score = np.mean(cross_val_score(classifier, train_word_features, train_target, scoring='roc_auc'))

    print('CV score for class {} is {}'.format(class_name, cv_score))
    scores.append(cv_score)
    
print('Total score is {}'.format(np.mean(scores)))



CV score for class toxic is 0.97193119880727
CV score for class severe_toxic is 0.9830390499432613
CV score for class obscene is 0.9840424143183912
CV score for class threat is 0.9859612844077553
CV score for class insult is 0.9760496985195051
CV score for class identity_hate is 0.9734729045817354
Total score is 0.979082758429653



```
C = 3.2
word_vectorizer = TfidfVectorizer()
Total score is 0.979082758429653
```

In [82]:
submission = pd.DataFrame.from_dict({'id': test['id']})

In [83]:
for class_name in class_names:  
    
    train_target = train[class_name]
    
    classifier = LogisticRegression(C=C,random_state=7)
      
    classifier.fit(train_word_features, train_target)
    
    submission[class_name] = classifier.predict_proba(test_word_features)[:, 1]

In [84]:
submission.to_csv('submission.csv')